In [26]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
trans = pd.read_csv('PayQwick pq_trans.csv')

In [29]:
trans

,CustomerId,TransactionDate,Type,SubType,Amount,SendorOrRecipient
0,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,3/31/22,Withdrawal,BillPayECheck,1246.81,A6A8F93C-745F-49A6-8ED3-8AAD663FBA52
1,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,3/30/22,Withdrawal,BillPay,1088.00,BFB98FD4-D715-4E80-9F5F-DB855B2FC37B
2,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,3/30/22,Withdrawal,BillPayECheck,3419.23,62D85FA8-BAB3-4A01-9ACF-9791AEF6B575
3,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,3/30/22,Withdrawal,BillPayECheck,14580.97,204C0AF0-02A2-4EAE-85A2-43A445887FE8
4,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,3/30/22,Deposit,ThirdPartyACH,203071.44,81528F49-3FD9-46E6-A329-952045061BBA
...,...,...,...,...,...,...
1805,0C58FEAD-771D-404F-8F6A-EBF602406CAD,9/9/21,Withdrawal,BillPay,2800.00,9DEF13C6-11CF-4BF6-A458-E524BC9FF49B
1806,0C58FEAD-771D-404F-8F6A-EBF602406CAD,9/9/21,Withdrawal,BillPay,3000.00,CE4645BD-A689-43B2-81AD-32DE7923C668
1807,0C58FEAD-771D-404F-8F6A-EBF602406CAD,9/9/21,Withdrawal,WireTransfer,5344.33,4810673E-ACDD-4B29-9F79-54DAABA71A5E
1808,0C58FEAD-771D-404F-8F6A-EBF602406CAD,9/9/21,Withdrawal,WireTransfer,9741.00,3DE8C4F3-0251-450B-A1D6-AAB43B5E4A8B


In [30]:
trans['TransactionDate'] = pd.to_datetime(trans['TransactionDate'])

In [31]:
trans['Amount'] = trans['Amount'].astype('float')

In [32]:
trans.head()

,CustomerId,TransactionDate,Type,SubType,Amount,SendorOrRecipient
0,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,2022-03-31,Withdrawal,BillPayECheck,1246.81,A6A8F93C-745F-49A6-8ED3-8AAD663FBA52
1,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,2022-03-30,Withdrawal,BillPay,1088.00,BFB98FD4-D715-4E80-9F5F-DB855B2FC37B
2,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,2022-03-30,Withdrawal,BillPayECheck,3419.23,62D85FA8-BAB3-4A01-9ACF-9791AEF6B575
3,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,2022-03-30,Withdrawal,BillPayECheck,14580.97,204C0AF0-02A2-4EAE-85A2-43A445887FE8
4,2B6C90C2-DA52-48C8-9CFD-1A23E0782526,2022-03-30,Deposit,ThirdPartyACH,203071.44,81528F49-3FD9-46E6-A329-952045061BBA


In [12]:
trans[['SendorOrRecipient','CustomerId','TransactionDate']].groupby(['SendorOrRecipient','TransactionDate']).count()

CustomerId
SendorOrRecipient                    TransactionDate            
00305698-D86E-4A6E-8F09-7A2405EC527D 1/15/22                   1
                                     10/15/21                  1
                                     11/15/21                  1
                                     11/5/21                   1
                                     12/15/21                  1
...                                                          ...
FDD2FA56-22B0-46A3-955D-C6BBC047FEE4 2/12/21                   2
                                     2/19/21                   1
                                     8/7/20                    1
FE65907C-E7CE-480F-96EC-66F49EA062E5 5/18/20                   1
FEF70388-0078-444E-B266-FAF91921AACC 1/13/22                   1

[1674 rows x 1 columns]

In [38]:
trans[(trans['CustomerId'] == 'D40352F4-AD58-4DD5-ABF2-3F27895A2CDE')&(trans['TransactionDate'] > '2022-02-28')]

,CustomerId,TransactionDate,Type,SubType,Amount,SendorOrRecipient
929,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-31,Deposit,WireTransfer,43050.00,271EC5BB-6743-45CA-83DF-7DDB5825BD4C
930,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-31,Withdrawal,ACHWithdrawal,234812.78,52562362-112B-4124-9221-FBFA34ED9F7A
931,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-23,Deposit,WireTransfer,100000.00,76408BCA-98FA-47A5-A599-200AF707BBFB
932,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-15,Withdrawal,BillPay,6733.08,81AA61AE-980C-448D-A56E-274E06A0535C
933,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-14,Withdrawal,WireTransfer,40856.75,F0258FE3-48F9-497D-82AD-ED65DD13C28A
934,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-14,Withdrawal,WireTransfer,50022.52,6F490B06-7E95-4C8E-BEF3-4BBAD3B54AC9
935,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-07,Withdrawal,WireTransfer,26150.00,46F254F7-68A9-4F66-8E0F-5DEFD6F0CC5A
936,D40352F4-AD58-4DD5-ABF2-3F27895A2CDE,2022-03-01,Withdrawal,ACHWithdrawal,608835.10,52562362-112B-4124-9221-FBFA34ED9F7A
